In [18]:
import os
import pydicom
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

In [ ]:
DATASET_PATH = "C:/Users/jrsl2/Downloads/archive/CT_low_dose_reconstruction_dataset"

def load_dicom_images(folder, size=(128, 128)):
    images = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith((".IMA", ".dcm")):  
                path = os.path.join(root, file)
                try:
                    dicom_data = pydicom.dcmread(path)
                    image = dicom_data.pixel_array
                    image = cv2.resize(image, size)
                    image = image.astype(np.float32) / np.max(image)  
                    images.append(image)
                except Exception as e:
                    print(f"Error {path}: {e}")
    print(f" {folder}: {len(images)}")
    return np.array(images)

original_data_path = os.path.join(DATASET_PATH, "Original Data")

quarter_dose_images = None
full_dose_images = None

for dose_type in ["Quarter Dose", "Full Dose"]:
    dose_path = os.path.join(original_data_path, dose_type)
    images = load_dicom_images(dose_path) 

    if dose_type == "Quarter Dose":
        quarter_dose_images = images
    else:
        full_dose_images = images 

print(f"{quarter_dose_images.shape}")
print(f"{full_dose_images.shape}")


quarter_dose_images = np.expand_dims(quarter_dose_images, axis=-1)
full_dose_images = np.expand_dims(full_dose_images, axis=-1)

print(f"{quarter_dose_images.shape}")
print(f"{full_dose_images.shape}")


Imagens Quarter Dose: (16628, 128, 128)
Imagens Full Dose: (16628, 128, 128)
Quarter Dose após expandir: (16628, 128, 128, 1)
Full Dose após expandir: (16628, 128, 128, 1)


In [ ]:
# Definir Autoencoder Convolucional
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2,2), padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), padding='same')(x)
    
    # Decoder
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)
    outputs = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)
    
    return Model(inputs, outputs)

# Criar e compilar o modelo
autoencoder = build_autoencoder((128, 128, 1))
autoencoder.compile(optimizer='adam', loss='mse')

# Treinar o modelo
autoencoder.fit(quarter_dose_images, full_dose_images, epochs=10, batch_size=8, shuffle=True)



In [ ]:
# Testar o modelo em uma imagem
def test_denoising(model, test_image):
    denoised_image = model.predict(np.expand_dims(test_image, axis=0))[0]
    return denoised_image

# Escolher uma imagem para teste
test_index = 5
test_image = quarter_dose_images[test_index]
clean_image = full_dose_images[test_index]
denosed_image = test_denoising(autoencoder, test_image)

# Avaliação com SSIM
similarity = ssim(clean_image.squeeze(), denosed_image.squeeze())
print(f"SSIM: {similarity:.4f}")

# Mostrar imagens
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(test_image.squeeze(), cmap='gray')
ax[0].set_title("Quarter Dose (ruidoso)")
ax[1].imshow(clean_image.squeeze(), cmap='gray')
ax[1].set_title("Full Dose (limpo)")
ax[2].imshow(denosed_image.squeeze(), cmap='gray')
ax[2].set_title("Após remoção de ruído")
plt.show()